In [41]:
import pandas as pd

In [14]:
import boto3
from io import BytesIO

In [8]:
files=['data/barrell BREAKING.csv','data/barrell FAST.csv','data/barrell OFFSPEED.csv','data/flare:burner BREAKING.csv','data/flare:burner FAST.csv',
       'data/flare:burner OFFSPEED.csv','data/no contact BREAKING.csv','data/no contact FAST.csv','data/no contact OFFSPEED.csv','data/poor:top BREAKING.csv',
      'data/poor:top OFFSPEED.csv','data/poor:top FAST.csv','data/poor:under BREAKING.csv','data/poor:under OFFSPEED.csv','data/poor:under FAST.csv',
      'data/poor:weak BREAKING.csv','data/poor:weak OFFSPEED.csv','data/poor:weak FAST.csv','data/solid contact BREAKING.csv','data/solid contact OFFSPEED.csv','data/solid contact FAST.csv']

In [38]:
def load_pitch_data_from_s3(csv_files, number_of_rows=None, bucket='qualitycontact'):
    '''
    Function to take a list of loan data CSV files that stored in an AWS S3 bucket and load and
    concatenate them into one dataframe.
    Args:
        csv_files (list or tuple): List of CSV files that contain the data. Below is a tuple of the names of the
        available files from Lending Club as of April 2019.
        ('LoanStats3a_securev1.csv', 'LoanStats3b_securev1.csv', 'LoanStats3c_securev1.csv', 'LoanStats3d_securev1.csv',
         'LoanStats_securev1_2016Q1.csv', 'LoanStats_securev1_2016Q2.csv', 'LoanStats_securev1_2016Q3.csv',
         'LoanStats_securev1_2016Q4.csv', 'LoanStats_securev1_2017Q1.csv', 'LoanStats_securev1_2017Q2.csv', 
         'LoanStats_securev1_2017Q3.csv', 'LoanStats_securev1_2017Q4.csv', 'LoanStats_securev1_2018Q1.csv',
         'LoanStats_securev1_2018Q2.csv', 'LoanStats_securev1_2018Q3.csv', 'LoanStats_securev1_2018Q4.csv')
        columns (list or tuple): List of column names that should be used in the dataframe. Certain columns should be
        excluded due to the fact they would not have been available at the time the loan was issued.
        The list of accetable columns is generated by the the file `columns.py` in the src folder and stored in 
        the variable columns_to_use.
        number_of_rows (int or None): The number of rows to load from each CSV file. This is used to load in smaller 
        amounts of data for testing purposes. By default, number_of_rows is None, which loads all data.
        bucket (string): Name of the S3 bucket the files are stored in. My bucket is called 'loan-analysis-data'.
    Returns:
        DataFrame: Returns a dataframe containing all loans contained within the list of CSV files.  
    '''
    datadict={}
    pitch_data = []
    for filename in csv_files:
        s3 = boto3.client('s3')
        obj = s3.get_object(Bucket=bucket, Key=filename)
        data = obj['Body'].read()
        f = BytesIO(data)
        data = pd.read_csv(f, low_memory=False, na_values='n/a', nrows=number_of_rows) 
        
        #Creating contact quality column and pitch type column
        datadict[filename]=data
        if filename in ['data/barrell BREAKING.csv','data/barrell FAST.csv','data/barrell OFFSPEED.csv','data/flare:burner BREAKING.csv','data/flare:burner FAST.csv','data/flare:burner OFFSPEED.csv', 'data/solid contact BREAKING.csv','data/solid contact OFFSPEED.csv','data/solid contact FAST.csv']:
            datadict[filename]['contact quality'] = 1
        else:
            datadict[filename]['contact quality'] = 0
            
        if filename in ['data/barrell FAST.csv','data/flare:burner FAST.csv','data/no contact FAST.csv','data/poor:top FAST.csv','data/poor:under FAST.csv','data/poor:weak FAST.csv','data/solid contact FAST.csv']:
            datadict[filename]['fast ball'] = 1
            datadict[filename]['offspeed'] = 0
            datadict[filename]['breaking'] = 0
        elif filename in ['data/barrell OFFSPEED.csv','data/flare:burner OFFSPEED.csv','data/no contact OFFSPEED.csv','data/poor:top OFFSPEED.csv','data/poor:under OFFSPEED.csv','data/poor:weak OFFSPEED.csv','data/solid contact OFFSPEED.csv']:
            datadict[filename]['fast ball'] = 0
            datadict[filename]['offspeed'] = 1
            datadict[filename]['breaking'] = 0
        else:
            datadict[filename]['fast ball'] = 0
            datadict[filename]['offspeed'] = 0
            datadict[filename]['breaking'] = 1
            
        pitch_data.append(datadict[filename])
    pitch = pd.concat(pitch_data)

    return pitch

In [39]:
pitch=load_pitch_data_from_s3(files)

In [40]:
pitch

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,contact quality,fast ball,offspeed,breaking
0,CU,2019-09-29,79.6,-1.9574,6.2977,Walker Lockett,588751,621141,home_run,hit_into_play_score,...,4,4,4,4,Standard,Standard,1,0,0,1
1,SL,2019-09-29,78.2,-2.2382,6.0700,Shane Greene,605412,572888,home_run,hit_into_play_score,...,3,3,3,3,Standard,Standard,1,0,0,1
2,SL,2019-09-29,84.0,2.0231,6.4508,Sam Howard,541650,607572,home_run,hit_into_play_score,...,2,0,2,0,Standard,Standard,1,0,0,1
3,SL,2019-09-29,86.5,-0.3244,6.8074,Yonny Chirinos,606192,630023,home_run,hit_into_play_score,...,1,2,2,1,Infield shift,Standard,1,0,0,1
4,KC,2019-09-29,80.0,-1.7519,5.9376,Jorge Lopez,488771,605347,home_run,hit_into_play_score,...,3,3,3,3,Standard,Standard,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4808,FC,2019-03-28,87.8,-1.4583,6.4755,Trevor Cahill,595777,502239,triple,hit_into_play_no_out,...,0,0,0,0,Standard,Standard,1,1,0,0
4809,FF,2019-03-28,92.7,-0.9541,6.5946,Andrew Cashner,609280,488768,field_out,hit_into_play,...,0,3,3,0,Standard,Standard,1,1,0,0
4810,FF,2019-03-28,92.2,-2.5181,5.0753,Julio Teheran,656555,527054,field_out,hit_into_play,...,0,1,1,0,Standard,Strategic,1,1,0,0
4811,FT,2019-03-28,95.4,-2.2056,5.9545,Jameson Taillon,458015,592791,double,hit_into_play_no_out,...,0,0,0,0,Standard,Standard,1,1,0,0


In [8]:
#Create copy of data to use for eda, keeping the original dataframe as is.
ss = df.copy()

In [10]:
ss.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [11]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 89 columns):
pitch_type                         39925 non-null object
game_date                          40000 non-null object
release_speed                      39813 non-null float64
release_pos_x                      39813 non-null float64
release_pos_z                      39813 non-null float64
player_name                        40000 non-null object
batter                             40000 non-null int64
pitcher                            40000 non-null int64
events                             14977 non-null object
description                        40000 non-null object
spin_dir                           0 non-null float64
spin_rate_deprecated               0 non-null float64
break_angle_deprecated             0 non-null float64
break_length_deprecated            0 non-null float64
zone                               39813 non-null float64
des                                14977 non-n

In [ ]:
ss.drop(['spin_dir','spin_rate_deprecated','break_angle_deprecated','break_length_deprecated','game_date','player_name','batter','pitcher','description','des','game_type'],axis=1)

In [17]:
ss[''].value_counts()

14.0    9929
13.0    5915
11.0    2831
12.0    2488
9.0     2481
2.0     2475
8.0     2445
5.0     2266
6.0     2002
1.0     1836
7.0     1772
3.0     1690
4.0     1683
Name: zone, dtype: int64

In [25]:
(ss['home_team']).value_counts()

BOS    1561
CIN    1483
LAA    1470
NYM    1436
DET    1421
MIL    1409
ARI    1407
MIA    1399
MIN    1394
PHI    1394
NYY    1388
CLE    1374
TB     1362
BAL    1345
STL    1334
SF     1333
TEX    1300
CHC    1288
CWS    1282
COL    1281
PIT    1274
TOR    1263
WSH    1244
LAD    1239
ATL    1233
SD     1231
OAK    1227
HOU    1222
KC     1204
SEA    1202
Name: home_team, dtype: int64

In [26]:
(ss['away_team']).value_counts()

CWS    1616
NYY    1548
DET    1539
TB     1431
CIN    1424
ATL    1419
LAD    1411
CHC    1399
TOR    1392
SD     1378
TEX    1363
MIN    1362
STL    1357
HOU    1343
MIA    1338
BAL    1317
SF     1316
PHI    1315
BOS    1314
WSH    1308
MIL    1287
NYM    1263
CLE    1256
COL    1235
PIT    1197
KC     1194
SEA    1186
OAK    1174
ARI    1164
LAA    1154
Name: away_team, dtype: int64